# **Especialização em Ciência de Dados - INF/UFRGS e SERPRO**
### Disciplina CD004 - Metodologia de Aprendizado de Máquina Supervisionado
#### *Profa. Mariana Recamonde-Mendoza (mrmendoza@inf.ufrgs.br)*
<br> 

---
***Observação:*** *Este notebook é disponibilizado aos alunos como complemento às aulas síncronas e aos slides preparados pela professora. Desta forma, os principais conceitos são apresentados no material teórico fornecido. O objetivo deste notebook é reforçar os conceitos e demonstrar questões práticas no uso de diferentes algoritmos e estratégias de Aprendizado de Máquina.*


---


<br>

## **Aula 05** - **Tópico: Pré-processamento de dados. Redução de dimensionalidade**

<br>

**Objetivo deste notebook**: Explorar estratégias para mitigar o problema de desbalanceamento de classes, complementando o pipeline da aula anterior que realizava pré-processamento dos dados através da imputação de valores e transformação de dados (codificação, discretização, normalização).
<br>

---





##**Predição de 'churn' de clientes de serviço de telecomunicação**

Os dados que utilizaremos neste notebook se referem a uma empresa de telecomunicações fictícia que forneceu serviços de telefone residencial e Internet para clientes na Califórnia. O conjunto de dados possui informações sobre os serviços para os quais cada cliente se inscreveu (telefone, várias linhas, internet, segurança online, backup online, proteção de dispositivos, suporte técnico e streaming de TV e filmes), informações da conta do cliente (há quanto tempo eles são clientes, contrato, forma de pagamento, cobrança sem papel, cobranças mensais e cobranças totais) e informações demográficas sobre os clientes (sexo, faixa etária, se eles têm parceiros e dependentes). Há, ainda, uma coluna chamada 'churn', que indica os clientes que desistiram do contrato do serviço no último mês. O objetivo da tarefa preditiva é identificar o churn (i.e., saída) de clientes a partir das informações coletadas. Os dados podem ser acessados neste [link](https://www.kaggle.com/datasets/blastchar/telco-customer-churn).

Este é um problema de predição que usualmente apresenta **classes desbalanceadas**. Neste notebook vamos adicionar ao pipeline de análise dos dados estratégias para tratar o desbalanceamento de classes.



---



In [ ]:
## Carregando as bibliotecas básicas necessárias
# A primeira linha é incluída para gerar os gráficos logo abaixo dos comandos de plot
%matplotlib inline              
import pandas as pd             # para análise de dados 
import matplotlib.pyplot as plt # para visualização de informações
import seaborn as sns           # para visualização de informações
import numpy as np              # para operações com arrays multidimensionais

###Carregando e inspecionando os dados

Primeiramente, vamos carregar algumas bibliotecas importantes do Python e os dados a serem utilizados neste estudo. Os dados são disponibilizados através de um link, que também pode ser diretamente acessado pelos alunos.

In [ ]:
## Bibliotecas para treinamento/avaliação de modelos
from sklearn.model_selection import RepeatedKFold, train_test_split, cross_validate, cross_val_score, GridSearchCV
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

sns.set()

In [ ]:
## Carregando os dados
df = pd.read_csv("https://drive.google.com/uc?export=view&id=10VrzI8mA2wPvkNIDaLzv-IglPU-Febtf")#,na_values="NA")
df  

In [ ]:
## Características gerais do dataset
print("O conjunto de dados possui {} linhas e {} colunas".format(df.shape[0], df.shape[1]))

A coluna *'Churn'* contém a classificação de cada instância. Vamos avaliar a distribuição de classes do problema.

In [ ]:
## Distribuição do atributo alvo
plt.hist(df['Churn'])
plt.title("Distribuição do atributo alvo")
plt.show()

Quais os tipos de atributos nos dados? Os dados possuem valores faltantes?
Podemos responder estas perguntas utilizando o método .info() para um dataframe.

In [ ]:
df.info()

O atributo customerID é único para cada instância e no geral não possui valor preditivo. Iremos removê-lo da análise.

In [ ]:
customer_ids = df['customerID']
df = df.drop(['customerID'], axis=1)

In [ ]:
##remove as duplicatas
df=df.drop_duplicates(keep='last')

Como atributos categóricos e numéricos podem demandar pré-processamentos diferentes, vamos separar os respectivos "nomes" em dois vetores distintos, facilitando a manipulação dos dados posteriormente.

In [ ]:
## Separa os atributos em vetores, de acordo com o tipo de dado (categórico ou numérico)
cat_columns=list(df.drop(['Churn'], axis=1).select_dtypes(include=["object"]).columns)
print(cat_columns)

num_columns=list(df.select_dtypes(include=["int64", "float64"]).columns)
print(num_columns)

Inspecionando a distribuição dos dados numéricos e categóricos:

In [ ]:
def dist_plot(df,columns,type='boxplot'):
    plt.figure(figsize=(16, 5))
    for indx, var  in enumerate(columns):
        plt.subplot(1, 3, indx+1)
        if (type=='boxplot'):
          g = sns.boxplot(x=var, data=df,showfliers=True)
        else: 
          if (type=='histogram'):
            g = sns.histplot(x=var, data=df)
    plt.tight_layout()

def count_plot(df,columns):
    plt.figure(figsize=(20, 12))
    for indx, var  in enumerate(columns):
        plt.subplot(6, 3, indx+1)
        g = sns.countplot(x=var, data=df)
    plt.tight_layout()

In [ ]:
count_plot(df,cat_columns)

In [ ]:
dist_plot(df,num_columns)#,type="histogram")


---


### Criando conjuntos de treino e teste para avaliação de modelos


Antes de iniciar o treinamento do modelo, lembre-se que é recomendado sempre reservar uma porção dos dados para teste, a qual somente será utilizada para avaliação do modelo final (após todo o processo de treinamento e otimização de hiperparâmetros).

Vamos fazer esta divisão, separando 20% para teste. Entretanto, primeiro precisamos dividir os dados entre atributos (X) e classe (y). Também iremos codificar as classes Yes/No para 1/0.



In [ ]:
## Separa o dataset em duas variáveis: os atributos/entradas (X) e a classe/saída (y)
X = df.drop(['Churn'], axis=1)
y = df['Churn'].values

Faremos o mapeamento das classes Yes/No para 1/0. Por padrão, as funções de avaliação assumem que a classe 1 é a positiva/de interesse.

In [ ]:
## substitui No' por 0, 'Yes' por 1
y = np.array([0 if y=='No' else 1 for y in y]) 

In [ ]:
## Faz a divisão entre treino (80%) e teste (20%).
## O conjunto de treino representa os dados que serão usados
## ao longo do desenvolvimento do modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,stratify=y,random_state=42) 

---


## Pré-Processamento dos Dados

Nesta seção, vamos realizar o pré-processamento dos dados com Pipelines. Em suma, os passos executados para cada tipo de atributo são:

*   Atributos **numéricos**: imputação de valores faltantes (com o método KNN) e normalização
*   Atributos **categóricos**: imputação de valores faltantes (com a moda), one-hot encoding, e normalização.

A normalização será feita pelo método StandardScaler (z-score), mais recomendado para uso com PCA. 

Salienta-se que como o objetivo deste notebook é observar os resultados dos métodos de redução de dimensionalidade, a aplicação destes métodos e o subsequente treinamento de modelos não será realizado com nested/k-fold cross-validation, mas sim com um simples holdout. Isto facilita a manipulação dos dados e a visualização dos seus resultados. Na prática, os métodos de redução de dimensionalidade devem ser inseridos no Pipeline que será executado a cada iteração do processo de treinamento e validação dos modelos.


In [ ]:
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

## pipeline específico para os atributos numéricos
num_pipeline = Pipeline([
                         ('imputer', KNNImputer(weights="uniform"))])

## pipeline específico para os atributos categóricos
cat_pipeline = Pipeline([
                         ('imputer', SimpleImputer(strategy='most_frequent')),
                         ('encoder', OneHotEncoder(drop='if_binary', sparse=False))])

## ColumnTransformer para aplicar cada pipeline ao respectivo tipo de atributo
data_pipeline = ColumnTransformer([
                                   ('numerical', num_pipeline, num_columns),
                                   ('categorical', cat_pipeline, cat_columns)])

## define pipeline que une as transformações definidas anteriormente e aplica a 
## normalização com método StandardScaler() em todos os atributos
prep_pipeline = Pipeline([
                 ('data_transform', data_pipeline),
                 ('data_normalize',StandardScaler())])

## ajusta o pipeline a partir dos dados de treino, e na sequência aplica em 
## treino e teste separadamente
prep_pipeline.fit(X_train)
X_train_prep = prep_pipeline.transform(X_train)
X_test_prep = prep_pipeline.transform(X_test)

In [ ]:
# ajusta nome das columas e mostra dataframe após pré-processamento
columns = np.append(num_columns,prep_pipeline[0].named_transformers_['categorical']['encoder'].get_feature_names_out(cat_columns))
df_train_prep = pd.DataFrame(X_train_prep, columns=columns)
df_test_prep = pd.DataFrame(X_test_prep, columns=columns)

Visualizando o formato do conjunto de dados de treinamento após aplicação do Pipeline de pré-processamento de dados.

In [ ]:
df_train_prep

In [ ]:
## Características gerais do dataset após pré-processamento
print("O conjunto de dados possui {} linhas e {} colunas referentes a atributos".format(df_train_prep.shape[0], df_train_prep.shape[1]))



---


## Redução de dimensionalidade com PCA

A aplicação do método PCA tem duas grandes utilidades com aprendizado supervisionado: auxiliar na visualização dos dados e reduzir a dimensionalidade dos dados para acelerar ou facilitar o treinamento de modelos preditivos.

### Compreendendo a saída do PCA

Nas células abaixo, vamos explorar o uso de PCA nos nossos dados sobre *Churn* de clientes. Os gráficos abaixo irão aplicar o PCA aos dados do treinamento, retornando as informações sobre a variância explicada pelas componentes principais. Execute as células, observe os resultados, e responda as perguntas a seguir.


In [ ]:
from sklearn.decomposition import PCA

#define modelo PCA a usar, sem restringir número de componentes principais para manter
pca = PCA()

#ajusta o modelo PCA aos dados (sem modificar os dados)
pca.fit(X_train_prep)

plt.figure(figsize=(10, 5))
PC_values = np.arange(pca.n_components_) + 1
plt.plot(PC_values, pca.explained_variance_ratio_, 'o-', linewidth=2, color='blue')
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.show()

In [ ]:
## descomente se quiser observar os valores das variâncias das componentes 1-11
print(pca.explained_variance_ratio_[0:10])

In [ ]:
plt.figure(figsize=(10, 5))
PC_values = np.arange(pca.n_components_) + 1
plt.plot(PC_values, np.cumsum(pca.explained_variance_ratio_), 'o-', linewidth=2, color='blue')
plt.title('Cumulative Variance Explained')
plt.xlabel('Principal Component')
plt.ylabel('Cumulative Variance Explained')
plt.show()

In [ ]:
## descomente se quiser observar os valores das variâncias cumulativas explicadas 
## pelas componentes 1-11
print(np.cumsum(pca.explained_variance_ratio_)[0:16])

**(1) Responda >>>** (i) Quanto da variância dos dados é explicado individualmente pela primeira e pela segunda componente? (ii) Quantas componentes são necessárias para explicar em torno de 90% da variância dos dados?


> ***Sua resposta aqui:***

(i) O primeiro componente PC1 captura 0.27949207 da variância nos dados, e o segundo componente PC2 captura 0.14413663 da variância.

(ii) Observando a variância acumulada dos componentes até o componente 16, temos a variância acumulada de 0.91792999.

### Visualizando os dados com PCA

In [ ]:
#projetando os dados de 40 dimensões para 2 (PC1 e PC2)
pca = PCA(2) 
X_train_projected = pca.fit_transform(X_train_prep)
X_test_projected = pca.transform(X_test_prep)
print(X_train_prep.shape)
print(X_train_projected.shape)

In [ ]:
fig, ax = plt.subplots()
scatter = ax.scatter(X_train_projected[:, 0], X_train_projected[:, 1],
            c=y_train, edgecolor='none', alpha=0.5,
            cmap=plt.cm.get_cmap('Dark2', 2))

legend1 = ax.legend(*scatter.legend_elements(),title="Classes")
ax.add_artist(legend1)

plt.xlabel('component 1')
plt.ylabel('component 2')


**(2) Responda >>>** (i) A projeção dos dados de acordo com as duas componentes principais apresenta algum tipo de agrupamento? Se sim, relate quantos grupos/clusters foram observado. (ii) Caso observe a formação de grupos, discuta se os grupos parecem ser divididos de acordo com a classe do problema (0/1) ou se não é esta característica que predomina na diferença entre os grupos.


> ***Sua resposta aqui:***
(i) Sim, foram observados 2 grupos um maior a esquerda do gráfico e um menor a direita.

(ii) Os grupos não parecem separar completamente as classes do problema (churn = 0 ou 1). Apesar do grupo menor visualmente apresentar poucos casos da classe 1, isso pode ser resultado do desbalanceamento das classes.

O PCA resulta de uma combinação linear entre os atributos originais. Assim, podemos observar qual o coeficiente associado com cada atributo para formar cada uma das componentes principais. A interpretação dos componentes principais baseia-se em descobrir quais atributos são mais fortemente correlacionadas com cada componente, ou seja, quais desses atributos possuem coeficientes de maior magnitude, em qualquer direção (positivo ou negativo), na componente analisada. Quanto maior o valor do coeficiente, mais importante ele é para o cálculo da componente. A decisão sobre que valor de coeficiente é grande ou pequeno é uma decisão subjetiva, por isso, uma análise relativa (entre atributos para uma mesma componente) pode ser mais útil em um primeiro momento.

Nas células a seguir, vamos extrair os coeficientes dos atributos no nosso conjunto de dados para cada uma das duas componentes principais. Em seguida, vamos visualizar os valores através de um heatmap (mapa de calor), facilitando a comparação entre atributos, e a identificação de diferentes características mais relevantes para cada componente

In [ ]:
loadings = pca.components_
num_pc = pca.n_features_
pc_list = ["PC"+str(i) for i in list(range(1, num_pc+1))]
loadings_df = pd.DataFrame.from_dict(dict(zip(pc_list, loadings)))
loadings_df['variable'] = df_train_prep.columns.values
loadings_df = loadings_df.set_index('variable')
loadings_df #descomente se quiser visualizar os dados brutos

In [ ]:
plt.figure(figsize=(10, 10))
sns.heatmap(loadings_df, annot=True, cmap='Spectral')
plt.show()

**(3) Responda >>>** Observe o heatmap e relate: (i) que atributos apresentaram maiores coeficientes para a PC1 e para a PC2? (ii) com base nesta análise, você consegue levantar alguma hipótese sobre a principal característica capturada pela PC1 e que pode explicar o resultado da visualização dos dados com base na PC1 e na PC2 descrito na questão (2)?

> ***Sua resposta aqui:***

(i)

Atributos com maiores correlação positiva ou negativa com atributo PC1 são: [InternetService_No
OnlineSecurity_No internet service
OnlineBackup_No internet service
DeviceProtection_No internet service
TechSupport_No internet service
StreamingTV_No internet service
StreamingMovies_No internet service
MonthlyCharges].

Atributos com maiores correlação positiva ou negativa com atributo PC2 são: [Contract_Two year,
tenure,
Contract_Month-to-month, TotalCharges].

(ii)

Os atributos [InternetService, OnlineSecurity, OnlineBackup, DeviceProtection
TechSupport, StreamingTV, StreamingMovies, MonthlyCharges] quando tem a classe  'No internet service' apresentam alta correlaçao positiva com o componente PC1.
Que pode explicar a separação dos dois grupos da questão (2), visto que os dois grupos eram separados no eixo x (PC1).
Esses mesmos atributos com os valores 'No' apresentam correlação negativa com o componente PC2.



---


## Treinando um modelo preditivo a partir das componentes principais

Nesta seção, vamos comparar o desempenho de modelos preditivos treinados com os dados originais, e com as componentes derivadas do PCA. Utilizaremos as duas componentes principais (PC1 e PC2) e posteriormente as 10 componentes principais (que explicam em torno de 80% da variância nos dados). Utilizaremos como classificador base um SVM com kernel radial e termo de regularização C=0.1 e não abordaremos a tarefa de otimizar os hiperparâmetros do modelo, utilizando o holdout simples (de 2 vias) para fins de comparação de desempenho. Salienta-se que ambos os conjuntos de treino e teste já passaram pelos mesmos passos de pré-processamento, incluindo redução de dimensionalidade com PCA.

In [ ]:
from sklearn.svm import SVC

## treinando e avaliando com os dados originais
clf_original = SVC(kernel='rbf', C=0.1)
clf_original = clf_original.fit(X_train_prep,y_train)
clf_original_pred = clf_original.predict(X_test_prep)

## treinando e avaliando com duas componentes principais
clf_pca2 = SVC(kernel='rbf', C=0.1)
clf_pca2 = clf_pca2.fit(X_train_projected,y_train)
clf_pca2_pred = clf_pca2.predict(X_test_projected)

## fazendo a análise do PCA, extraindo 10 componentes principais
## usadas no treinamento e avaliação
pca10 = PCA(10) 
X_train_projected10 = pca10.fit_transform(X_train_prep)
X_test_projected10 = pca10.transform(X_test_prep)
clf_pca10 = SVC(kernel='rbf', C=0.1)
clf_pca10 = clf_pca10.fit(X_train_projected10,y_train)
clf_pca10_pred = clf_pca10.predict(X_test_projected10)


In [ ]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score,f1_score,ConfusionMatrixDisplay

cm = confusion_matrix(y_test, clf_original_pred, labels=clf_original.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf_original.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()
print("F1: {}".format(round(f1_score(y_test, clf_original_pred),3)))
print("Recall: {} ".format(round(recall_score(y_test, clf_original_pred),3)))
print("Precision: {}".format(round(precision_score(y_test, clf_original_pred),3)))

In [ ]:
cm = confusion_matrix(y_test, clf_pca2_pred, labels=clf_original.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf_original.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()
print("F1: {}".format(round(f1_score(y_test, clf_pca2_pred),3)))
print("Recall: {} ".format(round(recall_score(y_test, clf_pca2_pred),3)))
print("Precision: {}".format(round(precision_score(y_test, clf_pca2_pred),3)))

In [ ]:
cm = confusion_matrix(y_test, clf_pca10_pred, labels=clf_pca10.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf_pca10.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()
print("F1: {}".format(round(f1_score(y_test, clf_pca10_pred),3)))
print("Recall: {} ".format(round(recall_score(y_test, clf_pca10_pred),3)))
print("Precision: {}".format(round(precision_score(y_test, clf_pca10_pred),3)))

**(4) Responda >>>** Qual o desempenho dos modelos SVM treinados sem e com PCA? Qual foi o impacto do uso do PCA sobre o desempenho do modelo (neste cenário) e como o aumento do número de componentes variou o desempenho?

> ***Sua resposta aqui:*** O desempenho dos modelos SVM com PCA teve resultados próximos, mas não tão bons quanto o modelo sem PCA. Mas houve melhora nas métricas com o aumento no número de componentes de 2 para 10.

Salienta-se que a resposta para a pergunta acima não necessariamente se aplica a outros problemas de predição. Isto é, a resposta é válida para o problema e para os dados analisados, e as observações podem variar para outros conjuntos de dados.



---



## Redução de dimensionalidade com seleção de atributos por Filtro

O módulo [sklearn.feature_selection](https://scikit-learn.org/stable/modules/feature_selection.html) possui uma série de funções para selecionar atributos, removendo aqueles que parecem não ser relevantes para uma determinada tarefa preditiva. As seções a seguir avaliam as estratégias baseadas em filtro usando o método SelectKBest(). Este método aplica um critério para ordenação dos atributos de acordo com sua relevância ou poder discriminativo e então retém apenas um subconjunto dos mais relevantes, de acordo com valor 'k' informado na chamada ao método.

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

## define o método de seleção de atributos (SelectKBest), critério para ordenar
## os atributos (F_classif),e  número de atributos para selecionar (2)
## Em seguida faz ajustae nos dados e transforma treino e teste
fsel_fclassif = SelectKBest(f_classif, k=2)
X_train_fsc = fsel_fclassif.fit_transform(X_train_prep, y_train)
X_test_fsc = fsel_fclassif.transform(X_test_prep)
print(X_train_fsc)
print("O conjunto de dados possui {} linhas e {} colunas".format(X_train_fsc.shape[0], X_train_fsc.shape[1]))

## encontra os índices das colunas de atributos selecionados
cols = fsel_fclassif.get_support(indices=True)
features_df_new = df_train_prep.iloc[:,cols]
print("Atributos selecionados: ")
print(columns[cols])

In [ ]:
clf_featSel_Fclassif = SVC(kernel='rbf', C=0.1)
clf_featSel_Fclassif = clf_featSel_Fclassif.fit(X_train_fsc,y_train)
clf_featSel_Fclassif_pred = clf_featSel_Fclassif.predict(X_test_fsc)

cm = confusion_matrix(y_test, clf_featSel_Fclassif_pred, labels=clf_featSel_Fclassif.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf_featSel_Fclassif.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()
print("F1: {}".format(round(f1_score(y_test, clf_featSel_Fclassif_pred),3)))
print("Recall: {} ".format(round(recall_score(y_test, clf_featSel_Fclassif_pred),3)))
print("Precision: {}".format(round(precision_score(y_test, clf_featSel_Fclassif_pred),3)))

**(5) Responda >>>** (i) Quais foram os top 2 atributos selecionados pelo método ANOVA F-value (F-Classif)? (ii) Qual foi o desempenho do modelo treinado com os 2 melhores atributos de acordo com o método F-Classif? (iii) Comparando recall e precisão, os valores obtidos com este modelo utilizando seleção de atributos (e top 2 atributos) foram acima ou abaixo do desempenho obtido com o modelo que usa apenas as duas primeiras componentes do PCA?

> ***Sua resposta aqui:***

(i) Os dois atributos selecionados foram: ['OnlineSecurity_No' 'Contract_Month-to-month'].

(ii) 

Resultado com seleção de atributos:

*   F1: 0.575
*   Recall: 0.704 
*   Precision: 0.485

Resultado dos top 2 atributos PCA:

*   F1: 0.454
*   Recall: 0.382 
*   Precision: 0.561

Os resultados com a seleção de algoritmo ANOVA F-value (F-Classif) foram superiores para as métricas de F1 e Recall.




Para fins de comparação, podemos verificar quais são os top 10 atributos de acordo com o método `f_classif` e utilizar somente estes para treinamento de um modelo.

In [ ]:
fsel_fclassif10 = SelectKBest(f_classif, k=10)
X_train_fsc10 = fsel_fclassif10.fit_transform(X_train_prep, y_train)
X_test_fsc10 = fsel_fclassif10.transform(X_test_prep)
#print(X_train_fsc10)
print("O conjunto de dados possui {} linhas e {} colunas".format(X_train_fsc10.shape[0], X_train_fsc10.shape[1]))

cols = fsel_fclassif10.get_support(indices=True)
features_df_new = df_train_prep.iloc[:,cols]
print("Atributos selecionados: ")
print(columns[cols])

In [ ]:
clf_featSel_Fclassif10 = SVC(kernel='rbf', C=0.1)
clf_featSel_Fclassif10 = clf_featSel_Fclassif10.fit(X_train_fsc10,y_train)
clf_featSel_Fclassif10_pred = clf_featSel_Fclassif10.predict(X_test_fsc10)

cm = confusion_matrix(y_test, clf_featSel_Fclassif10_pred, labels=clf_featSel_Fclassif10.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf_featSel_Fclassif10.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()
print("F1: {}".format(round(f1_score(y_test, clf_featSel_Fclassif10_pred),3)))
print("Recall: {} ".format(round(recall_score(y_test, clf_featSel_Fclassif10_pred),3)))
print("Precision: {}".format(round(precision_score(y_test, clf_featSel_Fclassif10_pred),3)))

Podemos observar desempenhos diferentes entre um modelo treinado a partir das 10 componentes principais obtidas pelo PCA e a partir dos top 10 atributos selecionados com um método de filtro (F-classif).

**(6) Responda >>>** Copie os códigos acima da seleção de atributos com SelectKBest e faça modificações para utilizar o método mutual_info_classif (ao invés de f_classif) para avaliação e priorização de atributos a serem selecionados. Treine e avalie o modelo para 2 e 10 atributos. Após a conclusão da implementação, execute o código e comente na sua resposta sobre: i) como o desempenho destes modelos se compara aos modelos com seleção de atributos com SelectKBest e método F-Classif e ii) quais atributos foram selecionados em cada casa e se ocorreu diferenças entre os métodos ou não quanto ao top K atributos  (para cada tamanho de subconjunto)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

In [ ]:
## >>> Sua resposta para a implementação 

# fsel_mutualclassif2
fsel_mutualclassif2 = SelectKBest(mutual_info_classif, k=2)
X_train_fsmutualclassif2 = fsel_mutualclassif2.fit_transform(X_train_prep, y_train)
X_test_fsmutualclassif2 = fsel_mutualclassif2.transform(X_test_prep)
print("O conjunto de dados possui {} linhas e {} colunas".format(X_train_fsmutualclassif2.shape[0], X_train_fsmutualclassif2.shape[1]))

cols = fsel_mutualclassif2.get_support(indices=True)
features_df_new = df_train_prep.iloc[:,cols]
print("Atributos selecionados: ")
print(columns[cols])


# fsel_mutualclassif10
fsel_mutualclassif10 = SelectKBest(mutual_info_classif, k=10)
X_train_fsmutualclassif10 = fsel_mutualclassif10.fit_transform(X_train_prep, y_train)
X_test_fsmutualclassif10 = fsel_mutualclassif10.transform(X_test_prep)
print("O conjunto de dados possui {} linhas e {} colunas".format(X_train_fsmutualclassif10.shape[0], X_train_fsmutualclassif10.shape[1]))

cols = fsel_mutualclassif10.get_support(indices=True)
features_df_new = df_train_prep.iloc[:,cols]
print("Atributos selecionados: ")
print(columns[cols])

In [ ]:
print('Matriz de confusão para mutual_info_classif k = 2:')

clf_featSel_Mclassif2 = SVC(kernel='rbf', C=0.1)
clf_featSel_Mclassif2 = clf_featSel_Mclassif2.fit(X_train_fsmutualclassif2,y_train)
clf_featSel_Mclassif2_pred = clf_featSel_Mclassif2.predict(X_test_fsmutualclassif2)

cm = confusion_matrix(y_test, clf_featSel_Mclassif2_pred, labels=clf_featSel_Mclassif2.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf_featSel_Mclassif2.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()
print("F1: {}".format(round(f1_score(y_test, clf_featSel_Mclassif2_pred),3)))
print("Recall: {} ".format(round(recall_score(y_test, clf_featSel_Mclassif2_pred),3)))
print("Precision: {}".format(round(precision_score(y_test, clf_featSel_Mclassif2_pred),3)))

print('Matriz de confusão para mutual_info_classif k = 10:')

clf_featSel_Mclassif10 = SVC(kernel='rbf', C=0.1)
clf_featSel_Mclassif10 = clf_featSel_Mclassif10.fit(X_train_fsmutualclassif10,y_train)
clf_featSel_Mclassif10_pred = clf_featSel_Mclassif10.predict(X_test_fsmutualclassif10)

cm = confusion_matrix(y_test, clf_featSel_Mclassif10_pred, labels=clf_featSel_Mclassif10.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf_featSel_Mclassif10.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()
print("F1: {}".format(round(f1_score(y_test, clf_featSel_Mclassif10_pred),3)))
print("Recall: {} ".format(round(recall_score(y_test, clf_featSel_Mclassif10_pred),3)))
print("Precision: {}".format(round(precision_score(y_test, clf_featSel_Mclassif10_pred),3)))

> ***Sua resposta aqui (para a interpretação dos resultados):***

(i)

Resultados para seleção de atributos com mutual_info_classif

*   k = 2: [F1: 0.505, Recall: 0.484, Precision: 0.528]
*   k = 10: [F1: 0.517, Recall: 0.441, Precision: 0.626]

Resultados para seleção de atributos com f_classif
*   k = 2: [F1: 0.575, Recall: 0.704, Precision: 0.485]
*   k = 10: [F1: 0.521, Recall: 0.441, Precision: 0.636]

Os dois métodos de escolha de atributos apresentaram uma diferença considerável para o caso k = 2, e uma diferença muito pequena no caso k = 10. Isso pode ser explicada analisando os atributos selecionados, como veremos a seguir.

ii) 

Atributos selecionados com mutual_info_classif:
*   top 2: ['tenure' 'Contract_Month-to-month']
*   top 10: ['tenure' 'MonthlyCharges' 'TotalCharges' 'InternetService_Fiber optic' 'OnlineSecurity_No' 'OnlineSecurity_No internet service' 'TechSupport_No'
 'Contract_Month-to-month' 'Contract_Two year' 'PaymentMethod_Electronic check']

Atributos selecionados com f_classif:
*   top 2: ['OnlineSecurity_No' 'Contract_Month-to-month']
*   top 10: ['tenure' 'InternetService_Fiber optic' 'OnlineSecurity_No'
 'OnlineBackup_No' 'DeviceProtection_No' 'TechSupport_No'
 'StreamingMovies_No internet service' 'Contract_Month-to-month'
 'Contract_Two year' 'PaymentMethod_Electronic check']

 Para o top 1, houve diferença de 1 dos elementos o que explica a diferença de performance dos dois modelos.
 Enquanto que no top 10, essa diferença foi só de 3 atributos dos 10 selecionados.
 É possível perceber que a performance do modelo varia mais quando poucos atributos são selecionados, que a diferença diminui a medida que esse grupo cresce.